In [7]:
import json
from pathlib import Path
from datetime import datetime
import pandas as pd
import sklearn 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from pandas.plotting import scatter_matrix
import seaborn as sns
%matplotlib inline

In [6]:
def clean_raw_data(data_file):
    raw_data=pd.read_csv(data_file)


In [ ]:
            #date=datetime.utcfromtimestamp(properties['time'] / 1000)
            #full_place=properties['place']
            #split_data=full_place.split(",")
            #if len(split_data)==2:
            #    country_sea=str(split_data[1])
            #else:
            #    country_sea=split_data[0]
            #if "Earthquake" in country_sea:
            #    country_sea.replace("Earthquake","")
            #country_sea=" ".join([word for word in country_sea.split() if word.istitle()])
            